In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from heapq import *
import random
import copy
total_nodes = 50

def get_distance(loc1, loc2, edges):
  queue = [loc1]
  dist = {}
  for i in range(total_nodes):
    dist[i+1] = 100
  dist[loc1] = 0
  while(queue):
    loc = queue.pop(0)
    if(loc == loc2):
      return dist[loc2]
    for neigh in edges[loc]:
      if(dist[neigh] > dist[loc] + 1):
        dist[neigh] = dist[loc] + 1
        queue.append(neigh)
  return dist[loc2]

def move_predator(env , predator_loc,agent_loc):
  choices = env.edges[predator_loc]
  dist = {}
  for choice in choices:
    d = get_distance(choice, agent_loc, env.edges)
    dist[choice] = d
  dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
  #print("For predator dist is ", dist)
  pred_choices = []
  for key, value in dist.items():
    if value == list(dist.values())[0]:
      pred_choices.append(key)
  predator_location = np.random.choice(pred_choices)
  return predator_location

def easily_distracted_predator(env , predator_loc, agent_loc):
  choices = env.edges[predator_loc]
  dist = {}
  for choice in choices:
    d = get_distance(choice, agent_loc, env.edges)
    dist[choice] = d
  dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
  #print("For easily distracted predator dist is ", dist)
  pred_optimal_choices = []
  neighbor_choices = []
  for key, value in dist.items():
    if value == list(dist.values())[0]:
      pred_optimal_choices.append(key)
    neighbor_choices.append(key)
  if random.random() <= 0.4:
    predator_location = np.random.choice(neighbor_choices)
  else:
    predator_location = np.random.choice(pred_optimal_choices)
  #print("distracted predator choose ", predator_location)
  return predator_location

def move_prey(env , prey_loc):
  choices = copy.deepcopy(env.edges[prey_loc])
  choices.append(prey_loc)
  prey_location = random.choice(choices)
  return prey_location

In [2]:
class The_Environment():
    def __init__(self):
        self.visual = []
        self.vertex = np.arange(1,total_nodes+1)
        self.edges = {}
        self.prey_location = 0
        self.predator_location = 0
        self.agent_location = 0
        self.the_environment()

    def addEdge(self, u, v):
        self.edges[u].append(v)
        self.visual.append([u, v])
          
    def visualize(self):
        G = nx.Graph()
        G.add_edges_from(self.visual)
        nx.draw_networkx(G)
        plt.show()
    
    def generate_ppa(self):
      self.prey_location = self.generate_location()
      self.predator_location = self.generate_location()
      self.agent_location = self.generate_location()
      while(self.agent_location == self.predator_location):
        self.agent_location = self.generate_location()
      while(self.agent_location == self.prey_location):
        self.agent_location = self.generate_location()

    def init_graph(self):
      for i in range(1,total_nodes+1):
        self.edges[i] = []
        if i ==1 : 
          self.addEdge(i,total_nodes)
          self.addEdge(i,i+1)
        elif i == total_nodes:
          self.addEdge(i,1)
          self.addEdge(i,i-1)
        else: 
          self.addEdge(i,i+1)
          self.addEdge(i,i-1)
      self.generate_ppa()
    
    def add_random_edges(self):
      for v in self.edges:
        #print("random edge for v: ", v)
        if(len(self.edges[v]) >= 3):
          continue
        choices = []
        for i in range(2, 6):
          u = v + i
          if(u > total_nodes):
            u = u%total_nodes
          choices.append(u)
        for i in range(-5, -1):
          u = v + i
          if(u < 0):
            u = (u+total_nodes)%total_nodes
          if(u == 0):
            u = total_nodes
          choices.append(u)
        #print("Initial choices aer ", choices)
        remove_choices = []
        for choice in choices:
          #print("for choice , ", choice, "edges is ", self.edges[choice])
          if(len(self.edges[choice]) >= 3):
            #print("removed")
            remove_choices.append(choice)
        choices = [choice for choice in choices if choice not in remove_choices]
        #print("Final choices ", choices)
        if not choices:
          continue
        random_choice = random.choice(choices)
        self.edges[v].append(random_choice)
        self.edges[random_choice].append(v)

    def the_environment(self):
      self.init_graph()
      self.add_random_edges()
    
    def generate_location(self):
      return np.random.randint(1, total_nodes+1)

In [3]:
def get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc):
  agent_choice = None
  priority = {}
  for i in pred_locs:
    if pred_locs[i] > pred_distance and prey_locs[i] < prey_distance:
      priority[i] = 1
      continue
    if pred_locs[i] == pred_distance and prey_locs[i] < prey_distance:
      priority[i] = 2
      continue
    if pred_locs[i] > pred_distance and prey_locs[i] == prey_distance:
      priority[i] = 3
      continue
    if pred_locs[i] == pred_distance and prey_locs[i] == prey_distance:
      priority[i] = 4
      continue
    if pred_locs[i] > pred_distance:
      priority[i] = 5
      continue
    if pred_locs[i] == pred_distance:
      priority[i] = 6
      continue
    else:
      priority[i] = 7
  priority = {k: v for k, v in sorted(priority.items(), key=lambda item: item[1])}
  choices = []
  if list(priority.values())[0] == 7:
    return agent_loc
  for key, value in priority.items():
    if value == list(priority.values())[0]:
      choices.append(key)
  agent_choice = random.choice(choices)
  return agent_choice

def init_prey_probs(agent_loc, true_location = None):
  prey_prob = {}
  if true_location:
    for i in range(1, total_nodes+1):
      if(i == true_location):
        prey_prob[i] = 1
      else:
        prey_prob[i] = 0
    return prey_prob
  for i in range(1, total_nodes+1):
    if(i == agent_loc):
      prey_prob[i] = 0
    else:
      prey_prob[i] = 1/(total_nodes-1)
  return prey_prob

def choose_node_for_survey(prey_prob, predator=False, agent_loc = None, edges = None):
  prey_prob = {k: v for k, v in sorted(prey_prob.items(), key=lambda item: item[1], reverse=True)}
  choices = []
  if list(prey_prob.values())[0] == 1:
    # Agent is certain so not choosing anything
    return None
  for key, value in prey_prob.items():
    if value == list(prey_prob.values())[0]:
      choices.append(key)
  if not predator:
    agent_choice = random.choice(choices)
  else:
    distances = {}
    for choice in choices:
      distances[choice] = get_distance(agent_loc, choice, edges)
    distances = {k: v for k, v in sorted(distances.items(), key=lambda item: item[1])}
    choices = []
    for key, value in distances.items():
      if value == list(distances.values())[0]:
        choices.append(key)
    agent_choice = random.choice(choices)
  #print("Node choosen for surveying is ", agent_choice)
  return agent_choice

def update_prey_probs_by_survey_defective(prey_prob, true_prey_loc, survey_node):
  if survey_node == true_prey_loc:
    if random.random() <= 0.1:
      node_failure = 1 - prey_prob[survey_node]
      for i in range(1, total_nodes+1):
        if(i == survey_node):
          prey_prob[i] = 0
        else:
          prey_prob[i] = prey_prob[i]/node_failure
    else:
      for i in range(1, total_nodes+1):
        if(i == survey_node):
          prey_prob[i] = 1
        else:
          prey_prob[i] = 0
    return prey_prob
  else:
    node_failure = 1 - prey_prob[survey_node]
    for i in range(1, total_nodes+1):
      if(i == survey_node):
        prey_prob[i] = 0
      else:
        prey_prob[i] = prey_prob[i]/node_failure 
    #print("prey probs are ", prey_prob)
    return prey_prob

def update_prey_probs_by_survey_defective_account(prey_prob, true_prey_loc, survey_node):
  if survey_node == true_prey_loc:
    if random.random() <= 0.1:
      node_failure = 1*(1 - prey_prob[survey_node]) + 0.1*prey_prob[survey_node]
      for i in range(1, total_nodes+1):
        if(i == survey_node):
          prey_prob[i] = (prey_prob[i]*0.1)/node_failure
        else:
          prey_prob[i] = (prey_prob[i]*1)/node_failure
    else:
      for i in range(1, total_nodes+1):
        if(i == survey_node):
          prey_prob[i] = 1
        else:
          prey_prob[i] = 0
    return prey_prob
  else:
    node_failure = 1*(1 - prey_prob[survey_node]) + 0.1*prey_prob[survey_node]
    for i in range(1, total_nodes+1):
      if(i == survey_node):
        prey_prob[i] = (prey_prob[i]*0.1)/node_failure
      else:
        prey_prob[i] = (prey_prob[i]*1)/node_failure
    #print("Updated prey probs are ", prey_prob)
    return prey_prob
    
def update_prey_probs_for_agent_9(prey_prob, true_prey_loc, survey_node):
  if survey_node == true_prey_loc:
    if random.random() <= 0.1:
      node_failure = 1*(1 - prey_prob[survey_node]) + 0.1*prey_prob[survey_node]
      for i in range(1, total_nodes+1):
        if(i == survey_node):
          prey_prob[i] = (prey_prob[i]*0.1)/node_failure
        else:
          prey_prob[i] = (prey_prob[i]*1)/node_failure
    else:
      for i in range(1, total_nodes+1):
        if(i == survey_node):
          prey_prob[i] = 1
        else:
          prey_prob[i] = 0
    return prey_prob
  else:
    flag = false
    if (prey_prob[survey_node] > 0.20):
      flag = true
      node_failure = 1*(1 - prey_prob[survey_node])
    else:
      node_failure = 1*(1 - prey_prob[survey_node]) + 0.1*prey_prob[survey_node]
    for i in range(1, total_nodes+1):
      if(i == survey_node):
        if flag:
          prey_prob[i] = (prey_prob[i]*0.1)/node_failure
        else:
          prey_prob[i] = 0
      else:
        prey_prob[i] = (prey_prob[i]*1)/node_failure
    #print("Updated prey probs are ", prey_prob)
    return prey_prob

def update_prey_probs_by_survey(prey_prob, true_prey_loc, survey_node):
  if survey_node == true_prey_loc:
    for i in range(1, total_nodes+1):
      if(i == survey_node):
        prey_prob[i] = 1
      else:
        prey_prob[i] = 0
    return prey_prob
  else:
    node_failure = 1 - prey_prob[survey_node]
    for i in range(1, total_nodes+1):
      if(i == survey_node):
        prey_prob[i] = 0
      else:
        prey_prob[i] = prey_prob[i]/node_failure 
    #print("prey probs are ", prey_prob)
    return prey_prob

def update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice):
  if agent_choice == true_prey_loc:
    return prey_prob
  elif prey_prob[agent_choice] != 1:
    node_failure = 1 - prey_prob[agent_choice]
    #print("the prob of agent choice is ", prey_prob[agent_choice])
    for i in range(1, total_nodes+1):
      if(i == agent_choice):
        prey_prob[i] = 0
      else:
        prey_prob[i] = prey_prob[i]/node_failure
    return prey_prob

def update_prey_probs_by_prey_movement(prey_prob, edges):
  new_prey_prob = copy.deepcopy(prey_prob)
  for i in new_prey_prob:
    new_prey_prob[i] = 0
  for i in edges.keys():
    num_neighs = len(edges[i]) + 1
    for neigh in edges[i]:
      new_prey_prob[neigh] = new_prey_prob[neigh] + prey_prob[i]/num_neighs
    new_prey_prob[i] = new_prey_prob[i] + prey_prob[i]/num_neighs
  return new_prey_prob

def get_highest_prob(prey_prob, predator=False, agent_loc = None, edges = None):
  prey_prob_by_priority = {k: v for k, v in sorted(prey_prob.items(), key=lambda item: item[1], reverse=True)}
  choices = []
  for key, value in prey_prob_by_priority.items():
    if value == list(prey_prob_by_priority.values())[0]:
      choices.append(key)
  if not predator:
    agent_choice = random.choice(choices)
  else:
    distances = {}
    for choice in choices:
      distances[choice] = get_distance(agent_loc, choice, edges)
    distances = {k: v for k, v in sorted(distances.items(), key=lambda item: item[1])}
    choices = []
    for key, value in distances.items():
      if value == list(distances.values())[0]:
        choices.append(key)
    agent_choice = random.choice(choices)
  #print("Node choosen for surveying is ", agent_choice)
  return agent_choice

def update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges):
  pred_nodes = {}
  for i in pred_prob:
    if pred_prob[i] > 0:
      pred_nodes[i] = pred_prob[i]
    pred_prob[i] = 0
  #print("Predicting predator node locations : ", pred_nodes)
  for node in pred_nodes:
    pred_choices = []
    neighbor_choices = []
    choices = edges[node]
    dist = {}
    #print("Analysing probs for node ", node)
    for choice in choices:
      d = get_distance(choice, agent_loc, edges)
      dist[choice] = d
    dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
    for key, value in dist.items():
      if value == list(dist.values())[0]:
        pred_choices.append(key)
      neighbor_choices.append(key)
    optimal_choice_len = len(pred_choices)
    neighbor_choice_len = len(neighbor_choices)
    #print("Final pred choices that can take are ", pred_choices)
    #print("Final distracted choices that can take are ", distracted_choices)
    for i in pred_prob:
      if i in pred_choices:
        pred_prob[i] = pred_prob[i] + (0.6/optimal_choice_len)*pred_nodes[node]
      if i in neighbor_choices:
        pred_prob[i] = pred_prob[i] + (0.4/neighbor_choice_len)*pred_nodes[node]
  return pred_prob

def get_utility(pred_probs, prey_probs, agent_loc, true_prey_loc, true_pred_loc, edges, by=None):
  prey_prob = copy.deepcopy(prey_probs)
  pred_prob = copy.deepcopy(pred_probs)
  agent_choice = agent_loc
  if by == 'survey':
    pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
    if pred_survey_node:
      pred_prob = update_prey_probs_by_survey(pred_prob, true_pred_loc, pred_survey_node)
    else:
      prey_survey_node = choose_node_for_survey(prey_prob)
      if prey_survey_node:
        prey_prob = update_prey_probs_by_survey(prey_prob, true_prey_loc, prey_survey_node)
  else:
    pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
    prey_loc = get_highest_prob(prey_prob)
    agent_choices = edges[agent_loc]
    pred_distance = get_distance(agent_loc, pred_loc, edges)
    prey_distance = get_distance(agent_loc, prey_loc, edges)
    prey_locs = {}
    pred_locs = {}
    for choice in agent_choices:
      prey_locs[choice] = get_distance(choice, prey_loc, edges)
      pred_locs[choice] = get_distance(choice, pred_loc, edges)
    agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
    prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
    pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
  pred_weighted_dist = 0
  prey_weighted_dist = 0
  for i in pred_prob:
    pred_weighted_dist += get_distance(agent_choice, i, edges)*pred_prob[i]
    prey_weighted_dist += get_distance(agent_choice, i, edges)*prey_prob[i]
  cost = prey_weighted_dist/pred_weighted_dist
  utility = 1/cost
  return utility

In [ ]:
class Agent(The_Environment):
  def __init__(self):
    pass

  def agent_1(self, env , init_step_count):
    agent_loc = env.agent_location
    prey_loc = env.prey_location
    pred_loc = env.predator_location
    edges = env.edges
    step_count = 0
    while(True):
      #print("Agent, prey and predator locations are ", agent_loc, prey_loc, pred_loc)
      if(prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count 
      if(agent_loc == pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      agent_choices = edges[agent_loc]
      #print("Agent choices ", agent_choices)
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        #print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          prey_locs[choice] = get_distance(choice, prey_loc, edges)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        #print("Agent choices by prey ", prey_locs)
        #print("Agent choices by pred ", pred_locs)
        #print("Prey and pred distance is  ", prey_distance, "and ", pred_distance)
        agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
        agent_loc = agent_choice
        #print("Agent choice is ", agent_choice)
      step_count = step_count + 1
      if(step_count == init_step_count):
        #print("Agent died because of timeout")
        return False, True, 0
      if(agent_loc == pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if(prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      prey_loc = move_prey(env,prey_loc)
      pred_loc = move_predator(env,pred_loc,agent_loc)

  def agent_2(self, env,init_step_count):
    agent_loc = env.agent_location
    prey_loc = env.prey_location
    pred_loc = env.predator_location
    edges = copy.deepcopy(env.edges)
    step_count = 0
    while(True and step_count <= init_step_count ):
      #print("Agent, prey and predator locations are ", agent_loc, prey_loc, pred_loc)
      if(prey_loc == agent_loc):
        print("Agent 2 caught the prey")
        return True , False , step_count
      if(agent_loc == pred_loc):
        print("Predator caught Agent 2")
        return False , False ,step_count
      agent_choices = edges[agent_loc]
      # print("Agent choices ", agent_choices)
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("Agent 2 found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          distance = 0
          if get_distance(choice, pred_loc, edges) < 2 : 
            continue
          prey_neighbours = edges[prey_loc]
          for prey_neighbr in prey_neighbours:
            distance += get_distance(choice,prey_neighbr, edges)
          prey_locs[choice] = distance/len(prey_neighbours)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        if len(prey_locs) > 0 :
          agent_loc = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
      if(agent_loc == pred_loc):
        print("Predator caught the agent")
        return False , False , step_count
      if(prey_loc == agent_loc):
        print("Agent caught the prey")
        return True , False , step_count
      prey_loc = move_prey(env,prey_loc)
      pred_loc = move_predator(env,pred_loc,agent_loc)
      step_count+=1
    print("Agent died because of timeout")
    return False , True , 0
    
  def agent_3(self, env,init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    prey_loc = None
    pred_loc = env.predator_location
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    #print("Initial prey probs are ", prey_prob)
    #print("True prey location is ", true_prey_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      survey_node = choose_node_for_survey(prey_prob)
      if survey_node:
        #print("Survey is chosen, updating probabilities by survey for ", survey_node)
        prey_prob = update_prey_probs_by_survey(prey_prob, true_prey_loc, survey_node)
        if 1 in list(prey_prob.values()):
          prey_certainity["agent_3"] = prey_certainity["agent_3"] + 1
      else:
        prey_certainity["agent_3"] = prey_certainity["agent_3"] + 1
      #print("prey probs after survey :", prey_prob)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      pred_distance = get_distance(agent_loc, pred_loc, edges)
      prey_distance = get_distance(agent_loc, prey_loc, edges)
      prey_locs = {}
      pred_locs = {}
      for choice in agent_choices:
        prey_locs[choice] = get_distance(choice, prey_loc, edges)
        pred_locs[choice] = get_distance(choice, pred_loc, edges)
      agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
      prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
      #print("Agent choice is ", agent_choice)
      #print("prey probs after agent movement :", prey_prob)
      agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        #print("Aborting since reached maxm steps")
        return False, True, 0
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      true_prey_loc = move_prey(env, true_prey_loc)
      #print("prey probs after prey movement :", prey_prob)
      #print("Sum of prey probs is ", find_sum(prey_prob))
      pred_loc = move_predator(env,pred_loc,agent_loc)
 
  def agent_4(self, env,init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    prey_loc = None
    pred_loc = env.predator_location
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    while(True and step_count <= init_step_count ):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False , step_count
      if(agent_loc == pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      survey_node = choose_node_for_survey(prey_prob)
      if survey_node:
        #print("Survey is chosen, updating probabilities by survey")
        prey_prob = update_prey_probs_by_survey(prey_prob, true_prey_loc, survey_node)
        if 1 in list(prey_prob.values()):
          prey_certainity["agent_4"] = prey_certainity["agent_4"] + 1
      #print("prey probs after survey :", prey_prob)
      else:
        prey_certainity["agent_4"] = prey_certainity["agent_4"] + 1
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      pred_distance = get_distance(agent_loc, pred_loc, edges)
      prey_distance = get_distance(agent_loc, prey_loc, edges)
      prey_locs = {}
      pred_locs = {}
      
      for choice in agent_choices:
        distance = 0
        probabolity_sum = 0
        
        if get_distance(choice, pred_loc, edges) < 2 : 
            continue
        prey_neighbours = edges[prey_loc]
        for prey_neighbr in prey_neighbours:
          if prey_prob[prey_neighbr] == 0 :
            distance += get_distance(choice,prey_neighbr, edges)
          else :
            distance += prey_prob[prey_neighbr]*get_distance(choice,prey_neighbr, edges)
          probabolity_sum += prey_prob[prey_neighbr]
        if probabolity_sum == 0 :
          prey_locs[choice] = distance/len(prey_neighbours)
        else:
          prey_locs[choice] = distance/probabolity_sum
        pred_locs[choice] = get_distance(choice, pred_loc, edges)
      agent_choice = agent_loc
      if len(prey_locs) >0:
        agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
      prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
      #print("prey probs after agent movement :", prey_prob)
      agent_loc = agent_choice
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_loc = move_predator(env,pred_loc,agent_loc)
      step_count+=1
    print("Agent died because of timeout")
    return False , True,0

  def agent_5(self, env , init_step_count):
    agent_loc = env.agent_location
    prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    edges = env.edges
    step_count = 0
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    #print("Initial pred probs are ", pred_prob)
    #print("True pred location is ", true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, prey_loc, true_pred_loc)
      if(prey_loc == agent_loc):
        print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        print("Predator caught the agent")
        return False, False, step_count
      survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if survey_node:
        #print("Survey node is chosen, updating probabilities by survey")
        pred_prob = update_prey_probs_by_survey(pred_prob, true_pred_loc, survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_5"] = predator_certainity["agent_5"] + 1
      else:
        predator_certainity["agent_5"] = predator_certainity["agent_5"] + 1
      #print("pred probs after survey :", pred_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      agent_choices = edges[agent_loc]
      pred_distance = get_distance(agent_loc, pred_loc, edges)
      prey_distance = get_distance(agent_loc, prey_loc, edges)
      prey_locs = {}
      pred_locs = {}
      for choice in agent_choices:
        prey_locs[choice] = get_distance(choice, prey_loc, edges)
        pred_locs[choice] = get_distance(choice, pred_loc, edges)
      agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
      pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
      #print("pred probs after agent movement :", pred_prob)
      agent_loc = agent_choice
      step_count = step_count + 1
      if(prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        print("Aborting since reached maxm steps")
        return False, True, 0
      prey_loc = move_prey(env, prey_loc)
      #print("Before update:, pred probs are ", pred_prob)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      #print("After update:, pred probs are ", pred_prob)
      #print("pred probs after pred movement :", pred_prob)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
      #print("Predator location is ", true_pred_loc)

  def agent_6_old(self, env , init_step_count):
    agent_loc = env.agent_location
    prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    edges = env.edges
    step_count = 0
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    print("Initial pred probs are ", pred_prob)
    #print("True pred location is ", true_pred_loc)
    while(True):
      print("Agent location, prey and predator location are ", agent_loc, prey_loc, true_pred_loc)
      if(prey_loc == agent_loc):
        print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        print("Predator caught the agent")
        return False, False, step_count
      survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if survey_node:
        print("Survey node is chosen, updating probabilities by survey")
        pred_prob = update_prey_probs_by_survey(pred_prob, true_pred_loc, survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_6"] = predator_certainity["agent_6"] + 1
      else:
        predator_certainity["agent_6"] = predator_certainity["agent_6"] + 1
      print("pred probs after survey :", pred_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          if get_distance(choice, pred_loc, edges) < 2 : 
              continue
          distance = 0
          prey_neighbours = edges[prey_loc]
          removed_prey_edge = copy.deepcopy(edges)
          for prey_neighbr in prey_neighbours:
            removed_prey_edge[prey_neighbr].remove(prey_loc)
          for prey_neighbr in prey_neighbours:
            distance += get_distance(choice,prey_neighbr, removed_prey_edge)
          prey_locs[choice] = distance/len(prey_neighbours)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = agent_loc
        if len(prey_locs) >0:
          #agent_choice =  min(prey_locs, key=prey_locs.get)
          agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        print("pred probs after agent movement :", pred_prob)
        agent_loc = agent_choice
        print("prey and pred locs are ", prey_locs, pred_locs)
      step_count = step_count + 1
      if(prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        print("Aborting since reached maxm steps")
        return False, True, 0
      prey_loc = move_prey(env, prey_loc)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      #print("pred probs after pred movement :", pred_prob)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)

  def agent_7(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    #print("Initial pred probs are ", pred_prob)
    #print("True pred location is ", true_pred_loc)
    #print("Initial prey probs are ", prey_prob)
    #print("True prey location is ", true_prey_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_7"] = predator_certainity["agent_7"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_7"] = predator_certainity["agent_7"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_7"] = prey_certainity["agent_7"] + 1
        else:
          prey_certainity["agent_7"] = prey_certainity["agent_7"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      pred_distance = get_distance(agent_loc, pred_loc, edges)
      prey_distance = get_distance(agent_loc, prey_loc, edges)
      prey_locs = {}
      pred_locs = {}
      for choice in agent_choices:
        prey_locs[choice] = get_distance(choice, prey_loc, edges)
        pred_locs[choice] = get_distance(choice, pred_loc, edges)
      agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
      #print("Agent choice is: ", agent_choice)
      pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
      prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
      #print("pred probs after agent movement :", pred_prob)
      #print("Prey probs after agent movement :", prey_prob)
      agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        #print("Aborting since reached maxm steps")
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
      #print("pred probs after pred movement :", pred_prob)
      #print("prey probs after prey movement :", prey_prob)

  def agent_7_defective_no_account(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey_defective(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_7_def_drone"] = predator_certainity["agent_7_def_drone"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_7_def_drone"] = predator_certainity["agent_7_def_drone"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey_defective(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_7_def_drone"] = prey_certainity["agent_7_def_drone"] + 1
        else:
          prey_certainity["agent_7_def_drone"] = prey_certainity["agent_7_def_drone"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      pred_distance = get_distance(agent_loc, pred_loc, edges)
      prey_distance = get_distance(agent_loc, prey_loc, edges)
      prey_locs = {}
      pred_locs = {}
      for choice in agent_choices:
        prey_locs[choice] = get_distance(choice, prey_loc, edges)
        pred_locs[choice] = get_distance(choice, pred_loc, edges)
      agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
      #print("Agent choice is: ", agent_choice)
      pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
      prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
      #print("pred probs after agent movement :", pred_prob)
      #print("Prey probs after agent movement :", prey_prob)
      agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        #print("Aborting since reached maxm steps")
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
      #print("pred probs after pred movement :", pred_prob)
      #print("prey probs after prey movement :", prey_prob)

  def agent_7_defective_account(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey_defective_account(pred_prob, true_pred_loc, pred_survey_node)
        #print("Updated pred probs are ", pred_prob)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_7_def_acc_drone"] = predator_certainity["agent_7_def_acc_drone"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_7_def_acc_drone"] = predator_certainity["agent_7_def_acc_drone"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey_defective_account(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_7_def_acc_drone"] = prey_certainity["agent_7_def_acc_drone"] + 1
          #print("Updated prey probs are ", prey_prob)
        else:
          prey_certainity["agent_7_def_acc_drone"] = prey_certainity["agent_7_def_acc_drone"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      pred_distance = get_distance(agent_loc, pred_loc, edges)
      prey_distance = get_distance(agent_loc, prey_loc, edges)
      prey_locs = {}
      pred_locs = {}
      for choice in agent_choices:
        prey_locs[choice] = get_distance(choice, prey_loc, edges)
        pred_locs[choice] = get_distance(choice, pred_loc, edges)
      agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
      #print("Agent choice is: ", agent_choice)
      pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
      prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
      #print("pred probs after agent movement :", pred_prob)
      #print("Prey probs after agent movement :", prey_prob)
      agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        #print("Aborting since reached maxm steps")
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
      #print("pred probs after pred movement :", pred_prob)
      #print("prey probs after prey movement :", prey_prob)
    
  def agent_8_old(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_8"] = predator_certainity["agent_8"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_8"] = predator_certainity["agent_8"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_8"] = prey_certainity["agent_8"] + 1
        else:
          prey_certainity["agent_8"] = prey_certainity["agent_8"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          distance = 0
          probabolity_sum = 0
          if get_distance(choice, pred_loc, edges) < 2 : 
              continue
          prey_neighbours = edges[prey_loc]
          for prey_neighbr in prey_neighbours:
            if prey_prob[prey_neighbr] == 0 :
              distance += get_distance(choice,prey_neighbr, edges)
            else :
              distance += prey_prob[prey_neighbr]*get_distance(choice,prey_neighbr, edges)
            probabolity_sum += prey_prob[prey_neighbr]
          if probabolity_sum == 0 :
            prey_locs[choice] = distance/len(prey_neighbours)
          else:
            prey_locs[choice] = distance/probabolity_sum
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = agent_loc
        if len(prey_locs) >0:
            #agent_choice =  min(prey_locs, key=prey_locs.get)
          agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
        #print("Agent choice is: ", agent_choice)
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        
        agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        return False, False, step_count
      if step_count == init_step_count:
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
  
  def agent_8_defective_no_account_old(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey_defective(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_8_def_drone"] = predator_certainity["agent_8_def_drone"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_8_def_drone"] = predator_certainity["agent_8_def_drone"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey_defective(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_8_def_drone"] = prey_certainity["agent_8_def_drone"] + 1
        else:
          prey_certainity["agent_8_def_drone"] = prey_certainity["agent_8_def_drone"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          if get_distance(choice, pred_loc, edges) < 2 : 
              continue
          prey_locs[choice] = get_distance(choice, prey_loc, edges)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = agent_loc
        if len(prey_locs) >0:
          #agent_choice =  min(prey_locs, key=prey_locs.get)
          agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
        #print("Agent choice is: ", agent_choice)
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        
        agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        return False, False, step_count
      if step_count == init_step_count:
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
  
  def agent_8_defective_account_old(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey_defective_account(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_8_def_acc_drone"] = predator_certainity["agent_8_def_acc_drone"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_8_def_acc_drone"] = predator_certainity["agent_8_def_acc_drone"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey_defective_account(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_8_def_acc_drone"] = prey_certainity["agent_8_def_acc_drone"] + 1
        else:
          prey_certainity["agent_8_def_acc_drone"] = prey_certainity["agent_8_def_acc_drone"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          if get_distance(choice, pred_loc, edges) < 2 : 
              continue
          prey_locs[choice] = get_distance(choice, prey_loc, edges)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = agent_loc
        if len(prey_locs) >0:
            #agent_choice =  min(prey_locs, key=prey_locs.get)
          agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
        #print("Agent choice is: ", agent_choice)
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        
        agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        return False, False, step_count
      if step_count == init_step_count:
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)

  def bonus_agent(self, env, init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      if(true_prey_loc == agent_loc):
        print("Prey entered Agent cell")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        print("Predator enetered agent cell")
        return False, False, step_count
      pred_prob = {k: v for k, v in sorted(pred_prob.items(), key=lambda item: item[1], reverse=True)}
      #print("prey predator and agent locs are ", true_prey_loc, true_pred_loc, agent_loc)
      
      u1 = get_utility(pred_prob, prey_prob, agent_loc, prey_loc, pred_loc, edges, by='suvrey')
      u2 = get_utility(pred_prob, prey_prob, agent_loc, prey_loc, pred_loc, edges, by='move')
      if 1 in list(pred_prob.values()):
        predator_certainity["bonus_agent"] = predator_certainity["bonus_agent"] + 1
      agent_choice = agent_loc
      if u1 > u2:
        #print("Utility of survey is more. Hence surveying.")
        pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
        if pred_survey_node:
          #print("Agent is not certain where the predator is, surveying for predator")
          pred_prob = update_prey_probs_by_survey_defective_account(pred_prob, true_pred_loc, pred_survey_node)
          if 1 in list(pred_prob.values()):
            predator_certainity["bonus_agent"] = predator_certainity["bonus_agent"] + 1
        else:
          #print("Agent is certain where the predator is, surveying for prey")
          predator_certainity["bonus_agent"] = predator_certainity["bonus_agent"] + 1
          prey_survey_node = choose_node_for_survey(prey_prob)
          #print("Node chosen for prey survey is  ", prey_survey_node)
          if prey_survey_node:
            prey_prob = update_prey_probs_by_survey_defective_account(prey_prob, true_prey_loc, prey_survey_node)
            if 1 in list(prey_prob.values()):
              prey_certainity["bonus_agent"] = prey_certainity["bonus_agent"] + 1
          else:
            prey_certainity["bonus_agent"] = prey_certainity["bonus_agent"] + 1
            #print("Agent is certain where the prey is")
      else:
        #print("Utility of movement is more. Hence moving.")
        pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
        prey_loc = get_highest_prob(prey_prob)
        agent_choices = edges[agent_loc]
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          prey_locs[choice] = get_distance(choice, prey_loc, edges)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = get_full_information_choice(prey_locs, pred_locs, prey_distance, pred_distance, agent_loc)
        #print("Agent choice is: ", agent_choice)
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        #print("pred probs after agent movement :", pred_prob)
        #print("Prey probs after agent movement :", prey_prob)
      agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        print("Agent entered predator cell")
        return False, False, step_count
      if step_count == init_step_count:
        print("Aborting since reached maxm steps")
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
      #print("pred probs after pred movement :", pred_prob)
      #print("prey probs after prey movement :", prey_prob)

  def agent_6(self, env , init_step_count):
    agent_loc = env.agent_location
    prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    edges = env.edges
    step_count = 0
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    #print("Initial pred probs are ", pred_prob)
    #print("True pred location is ", true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, prey_loc, true_pred_loc)
      if(prey_loc == agent_loc):
        print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        print("Predator caught the agent")
        return False, False, step_count
      survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if survey_node:
        #print("Survey node is chosen, updating probabilities by survey")
        pred_prob = update_prey_probs_by_survey(pred_prob, true_pred_loc, survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_6"] = predator_certainity["agent_6"] + 1
      else:
        predator_certainity["agent_6"] = predator_certainity["agent_6"] + 1
      #print("pred probs after survey :", pred_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        simulated_pred_loc = easily_distracted_predator(env, pred_loc, agent_loc)
        pred_distance = get_distance(agent_loc, simulated_pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        restricted_choices = []
        for choice in agent_choices:
          if get_distance(choice, pred_loc, edges) < 2 : 
              restricted_choices.append(choice)
              continue
          distance = 0
          prey_neighbours = edges[prey_loc]
          removed_prey_edge = copy.deepcopy(edges)
          for prey_neighbr in prey_neighbours:
            removed_prey_edge[prey_neighbr].remove(prey_loc)
          for prey_neighbr in prey_neighbours:
            distance += get_distance(choice,prey_neighbr, removed_prey_edge)
          prey_locs[choice] = distance/len(prey_neighbours)
        pred_choices = []
        neighbor_choices = []
        dist = {}
        for p_choice in edges[pred_loc]:
          d = get_distance(p_choice, agent_loc, edges)
          dist[p_choice] = d
        dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
        for key, value in dist.items():
          if value == list(dist.values())[0]:
            pred_choices.append(key)
          neighbor_choices.append(key)
        optimal_choice_len = len(pred_choices)
        neighbor_choice_len = len(neighbor_choices)
        simulate_prob = {}
        for i in edges[pred_loc]:
          simulate_prob[i] = 0
          if i in pred_choices:
            simulate_prob[i] += 0.6/optimal_choice_len
          if i in neighbor_choices:
            simulate_prob[i] += 0.4/neighbor_choice_len
        
        for choice in agent_choices:
          if choice in restricted_choices:
            continue
          d = 0
          for pred_choice in edges[pred_loc]:
              d += get_distance(pred_choice, choice, edges)*simulate_prob[pred_choice]
          pred_locs[choice] = d
        
        #print("prey and pred locs are ", prey_locs, pred_locs)
        agent_choice = agent_loc
        prey_locs = {k: v for k, v in sorted(prey_locs.items(), key=lambda item: item[1])}
        maxm = 0
        for i in prey_locs:
          if (pred_locs[i] > maxm):
            maxm = pred_locs[i]
            agent_choice = i
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        #print("pred probs after agent movement :", pred_prob)
        agent_loc = agent_choice
      step_count = step_count + 1
      if(prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        print("Aborting since reached maxm steps")
        return False, True, 0
      prey_loc = move_prey(env, prey_loc)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
  
  def agent_8(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_8"] = predator_certainity["agent_8"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_8"] = predator_certainity["agent_8"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_8"] = prey_certainity["agent_8"] + 1
        else:
          prey_certainity["agent_8"] = prey_certainity["agent_8"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      restricted_choices = []
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          distance = 0
          probabolity_sum = 0
          if get_distance(choice, pred_loc, edges) < 2 : 
              restricted_choices.append(choice)
              continue
          prey_neighbours = edges[prey_loc]
          for prey_neighbr in prey_neighbours:
            if prey_prob[prey_neighbr] == 0 :
              distance += get_distance(choice,prey_neighbr, edges)
            else :
              distance += prey_prob[prey_neighbr]*get_distance(choice,prey_neighbr, edges)
            probabolity_sum += prey_prob[prey_neighbr]
          if probabolity_sum == 0 :
            prey_locs[choice] = distance/len(prey_neighbours)
          else:
            prey_locs[choice] = distance/probabolity_sum
        
        agent_choice = agent_loc
        pred_choices = []
        neighbor_choices = []
        dist = {}
        for p_choice in edges[pred_loc]:
          d = get_distance(p_choice, agent_loc, edges)
          dist[p_choice] = d
        dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
        for key, value in dist.items():
          if value == list(dist.values())[0]:
            pred_choices.append(key)
          neighbor_choices.append(key)
        optimal_choice_len = len(pred_choices)
        neighbor_choice_len = len(neighbor_choices)
        simulate_prob = {}
        for i in edges[pred_loc]:
          simulate_prob[i] = 0
          if i in pred_choices:
            simulate_prob[i] += 0.6/optimal_choice_len
          if i in neighbor_choices:
            simulate_prob[i] += 0.4/neighbor_choice_len
        
        for choice in agent_choices:
          if choice in restricted_choices:
            continue
          d = 0
          for pred_choice in edges[pred_loc]:
              d += get_distance(pred_choice, choice, edges)*simulate_prob[pred_choice]
          pred_locs[choice] = d
        
        #print("prey and pred locs are ", prey_locs, pred_locs)
        agent_choice = agent_loc
        prey_locs = {k: v for k, v in sorted(prey_locs.items(), key=lambda item: item[1])}
        maxm = 0
        for i in prey_locs:
          if (pred_locs[i] > maxm):
            maxm = pred_locs[i]
            agent_choice = i
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        
        agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        print("Predator caught the agent")
        return False, False, step_count
      if step_count == init_step_count:
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)

  def agent_8_defective_no_account(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey_defective(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_8_def_drone"] = predator_certainity["agent_8_def_drone"] + 1
      else:
        restricted_choices = []
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_8_def_drone"] = predator_certainity["agent_8_def_drone"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey_defective(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_8_def_drone"] = prey_certainity["agent_8_def_drone"] + 1
        else:
          prey_certainity["agent_8_def_drone"] = prey_certainity["agent_8_def_drone"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          if get_distance(choice, pred_loc, edges) < 2 : 
              restricted_choices.append(choice)
              continue
          prey_locs[choice] = get_distance(choice, prey_loc, edges)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = agent_loc
        pred_choices = []
        neighbor_choices = []
        dist = {}
        for p_choice in edges[pred_loc]:
          d = get_distance(p_choice, agent_loc, edges)
          dist[p_choice] = d
        dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
        for key, value in dist.items():
          if value == list(dist.values())[0]:
            pred_choices.append(key)
          neighbor_choices.append(key)
        optimal_choice_len = len(pred_choices)
        neighbor_choice_len = len(neighbor_choices)
        simulate_prob = {}
        for i in edges[pred_loc]:
          simulate_prob[i] = 0
          if i in pred_choices:
            simulate_prob[i] += 0.6/optimal_choice_len
          if i in neighbor_choices:
            simulate_prob[i] += 0.4/neighbor_choice_len
        
        for choice in agent_choices:
          if choice in restricted_choices:
            continue
          d = 0
          for pred_choice in edges[pred_loc]:
              d += get_distance(pred_choice, choice, edges)*simulate_prob[pred_choice]
          pred_locs[choice] = d
        
        #print("prey and pred locs are ", prey_locs, pred_locs)
        agent_choice = agent_loc
        prey_locs = {k: v for k, v in sorted(prey_locs.items(), key=lambda item: item[1])}
        maxm = 0
        for i in prey_locs:
          if (pred_locs[i] > maxm):
            maxm = pred_locs[i]
            agent_choice = i

        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        
        agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        return False, False, step_count
      if step_count == init_step_count:
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)
  
  def agent_8_defective_account(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey_defective_account(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_8_def_acc_drone"] = predator_certainity["agent_8_def_acc_drone"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_8_def_acc_drone"] = predator_certainity["agent_8_def_acc_drone"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey_defective_account(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_8_def_acc_drone"] = prey_certainity["agent_8_def_acc_drone"] + 1
        else:
          prey_certainity["agent_8_def_acc_drone"] = prey_certainity["agent_8_def_acc_drone"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        restricted_choices = []
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          if get_distance(choice, pred_loc, edges) < 2 : 
              restricted_choices.append(choice)
              continue
          prey_locs[choice] = get_distance(choice, prey_loc, edges)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = agent_loc
        pred_choices = []
        neighbor_choices = []
        dist = {}
        for p_choice in edges[pred_loc]:
          d = get_distance(p_choice, agent_loc, edges)
          dist[p_choice] = d
        dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
        for key, value in dist.items():
          if value == list(dist.values())[0]:
            pred_choices.append(key)
          neighbor_choices.append(key)
        optimal_choice_len = len(pred_choices)
        neighbor_choice_len = len(neighbor_choices)
        simulate_prob = {}
        for i in edges[pred_loc]:
          simulate_prob[i] = 0
          if i in pred_choices:
            simulate_prob[i] += 0.6/optimal_choice_len
          if i in neighbor_choices:
            simulate_prob[i] += 0.4/neighbor_choice_len
        
        for choice in agent_choices:
          if choice in restricted_choices:
            continue
          d = 0
          for pred_choice in edges[pred_loc]:
              d += get_distance(pred_choice, choice, edges)*simulate_prob[pred_choice]
          pred_locs[choice] = d
        
        #print("prey and pred locs are ", prey_locs, pred_locs)
        agent_choice = agent_loc
        prey_locs = {k: v for k, v in sorted(prey_locs.items(), key=lambda item: item[1])}
        maxm = 0
        for i in prey_locs:
          if (pred_locs[i] > maxm):
            maxm = pred_locs[i]
            agent_choice = i
        #print("Agent choice is: ", agent_choice)
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        
        agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        return False, False, step_count
      if step_count == init_step_count:
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)

  def agent_9(self, env , init_step_count):
    agent_loc = env.agent_location
    true_prey_loc = env.prey_location
    true_pred_loc = env.predator_location
    pred_loc = true_pred_loc
    prey_loc = None
    edges = env.edges
    step_count = 0
    prey_prob = init_prey_probs(agent_loc)
    pred_prob = init_prey_probs(agent_loc, true_location=true_pred_loc)
    while(True):
      #print("Agent location, prey and predator location are ", agent_loc, true_prey_loc, true_pred_loc)
      if(true_prey_loc == agent_loc):
        #print("Agent caught the prey")
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        #print("Predator caught the agent")
        return False, False, step_count
      pred_survey_node = choose_node_for_survey(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      if pred_survey_node:
        #print("Agent is not certain where the predator is, surveying for predator")
        pred_prob = update_prey_probs_by_survey_defective_account(pred_prob, true_pred_loc, pred_survey_node)
        if 1 in list(pred_prob.values()):
          predator_certainity["agent_9"] = predator_certainity["agent_9"] + 1
      else:
        #print("Agent is certain where the predator is, surveying for prey")
        predator_certainity["agent_9"] = predator_certainity["agent_9"] + 1
        prey_survey_node = choose_node_for_survey(prey_prob)
        #print("Node chosen for prey survey is  ", prey_survey_node)
        if prey_survey_node:
          prey_prob = update_prey_probs_by_survey_defective_account(prey_prob, true_prey_loc, prey_survey_node)
          if 1 in list(prey_prob.values()):
            prey_certainity["agent_9"] = prey_certainity["agent_9"] + 1
        else:
          prey_certainity["agent_9"] = prey_certainity["agent_9"] + 1
          print("Agent is certain where the prey is")
      #print("pred probs after survey :", pred_prob)
      #print("prey probs after survey : ", prey_prob)
      pred_loc = get_highest_prob(pred_prob, predator=True, agent_loc=agent_loc, edges=edges)
      prey_loc = get_highest_prob(prey_prob)
      agent_choices = edges[agent_loc]
      if (prey_loc in agent_choices and prey_loc != pred_loc):
        print("found the prey in neighbour")
        agent_loc = prey_loc
      else:
        restricted_choices = []
        pred_distance = get_distance(agent_loc, pred_loc, edges)
        prey_distance = get_distance(agent_loc, prey_loc, edges)
        prey_locs = {}
        pred_locs = {}
        for choice in agent_choices:
          if get_distance(choice, pred_loc, edges) < 2 : 
              restricted_choices.append(choice)
              continue
          prey_locs[choice] = get_distance(choice, prey_loc, edges)
          pred_locs[choice] = get_distance(choice, pred_loc, edges)
        agent_choice = agent_loc
        pred_choices = []
        neighbor_choices = []
        dist = {}
        for p_choice in edges[pred_loc]:
          d = get_distance(p_choice, agent_loc, edges)
          dist[p_choice] = d
        dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
        for key, value in dist.items():
          if value == list(dist.values())[0]:
            pred_choices.append(key)
          neighbor_choices.append(key)
        optimal_choice_len = len(pred_choices)
        neighbor_choice_len = len(neighbor_choices)
        simulate_prob = {}
        for i in edges[pred_loc]:
          simulate_prob[i] = 0
          if i in pred_choices:
            simulate_prob[i] += 0.6/optimal_choice_len
          if i in neighbor_choices:
            simulate_prob[i] += 0.4/neighbor_choice_len
        
        for choice in agent_choices:
          if choice in restricted_choices:
            continue
          d = 0
          for pred_choice in edges[pred_loc]:
              d += get_distance(pred_choice, choice, edges)*simulate_prob[pred_choice]
          pred_locs[choice] = d
        
        #print("prey and pred locs are ", prey_locs, pred_locs)
        agent_choice = agent_loc
        prey_locs = {k: v for k, v in sorted(prey_locs.items(), key=lambda item: item[1])}
        maxm = 0
        for i in prey_locs:
          if (pred_locs[i] > maxm):
            maxm = pred_locs[i]
            agent_choice = i
        #print("Agent choice is: ", agent_choice)
        pred_prob = update_prey_probs_by_agent(pred_prob, true_pred_loc, agent_choice)
        prey_prob = update_prey_probs_by_agent(prey_prob, true_prey_loc, agent_choice)
        
        agent_loc = agent_choice
      step_count = step_count + 1
      if(true_prey_loc == agent_loc):
        return True, False, step_count
      if(agent_loc == true_pred_loc):
        return False, False, step_count
      if step_count == init_step_count:
        return False, True, 0
      true_prey_loc = move_prey(env, true_prey_loc)
      prey_prob = update_prey_probs_by_prey_movement(prey_prob, edges)
      pred_prob = update_pred_probs_by_pred_movement(pred_prob, agent_loc, edges)
      true_pred_loc = easily_distracted_predator(env, true_pred_loc, agent_loc)

In [ ]:
wins = {"agent_1":0, "agent_2":0, "agent_3":0, "agent_4":0, "agent_5":0, "agent_6":0, "agent_7":0, "agent_7_def_drone":0, "agent_7_def_acc_drone": 0, "agent_8":0, "agent_8_def_drone":0, "agent_8_def_acc_drone": 0, 'bonus_agent':0, 'agent_9':0}
timeouts = {"agent_1":0, "agent_2":0, "agent_3":0, "agent_4":0, "agent_5":0, "agent_6":0, "agent_7":0, "agent_7_def_drone":0, "agent_7_def_acc_drone": 0, "agent_8":0, "agent_8_def_drone":0, "agent_8_def_acc_drone": 0, 'bonus_agent':0, 'agent_9':0}
step_counts = {"agent_1":0, "agent_2":0, "agent_3":0, "agent_4":0, "agent_5":0, "agent_6":0, "agent_7":0, "agent_7_def_drone":0, "agent_7_def_acc_drone": 0, "agent_8":0, "agent_8_def_drone":0, "agent_8_def_acc_drone": 0, 'bonus_agent':0, 'agent_9':0}
prey_certainity = {'agent_7_def_acc_drone': 0,
 'agent_3': 0,
 'agent_4': 0,
 'agent_7': 0,
 'agent_7_def_drone': 0,
 'agent_8': 0,
 'agent_8_def_drone':0,
 'agent_8_def_acc_drone':0,
 'bonus_agent':0,
 'agent_9':0}
predator_certainity = {'agent_7_def_acc_drone': 0,
 'agent_5': 0,
 'agent_6': 0,
 'agent_7': 0,
 'agent_7_def_drone': 0,
 'agent_8': 0,
 'agent_8_def_drone':0,
 'agent_8_def_acc_drone':0,
 'bonus_agent':0,
 'agent_9':0}
step_count_odd = 75
step_count_even = 75
for i in range(0, 3000):
  env = The_Environment()
  agent = Agent()
  success, timeout, step_count = agent.agent_1(env, 50)
  if success:
    wins["agent_1"] = wins["agent_1"] +1
  if timeout:
    timeouts["agent_1"] = timeouts["agent_1"] + timeout
  step_counts["agent_1"] = step_counts["agent_1"] + step_count
 
  success , timeout , step_count = agent.agent_2(env , 50)
  if success:
    wins["agent_2"] = wins["agent_2"] +1
  if timeout:
    timeouts["agent_2"] = timeouts["agent_2"] + timeout
  step_counts["agent_2"] = step_counts["agent_2"] + step_count
 
  success, timeout, step_count = agent.agent_3(env, 100)
  if success:
    wins["agent_3"] = wins["agent_3"] +1
  if timeout:
    timeouts["agent_3"] = timeouts["agent_3"] + timeout
  step_counts["agent_3"] = step_counts["agent_3"] + step_count
 
  success, timeout, step_count  = agent.agent_4(env, 100)
  if success:
    wins["agent_4"] = wins["agent_4"] +1
  if timeout:
    timeouts["agent_4"] = timeouts["agent_4"] + timeout
  step_counts["agent_4"] = step_counts["agent_4"] + step_count

  success, timeout, step_count = agent.agent_5(env, 100)
  if success:
    wins["agent_5"] = wins["agent_5"] + 1
  if timeout:
    timeouts["agent_5"] = timeouts["agent_5"] + timeout
  step_counts["agent_5"] = step_counts["agent_5"] + step_count

  success, timeout, step_count = agent.agent_6(env, 100)
  if success:
    wins["agent_6"] = wins["agent_6"] + 1
  if timeout:
    timeouts["agent_6"] = timeouts["agent_6"] + timeout
  step_counts["agent_6"] = step_counts["agent_6"] + step_count

  success, timeout, step_count = agent.agent_7(env, 150)
  if success:
    wins["agent_7"] = wins["agent_7"] + 1
  if timeout:
    timeouts["agent_7"] = timeouts["agent_7"] + timeout
  step_counts["agent_7"] = step_counts["agent_7"] + step_count

  success, timeout, step_count = agent.agent_7_defective_no_account(env, 150)
  if success:
    wins["agent_7_def_drone"] = wins["agent_7_def_drone"] + 1
  if timeout:
    timeouts["agent_7_def_drone"] = timeouts["agent_7_def_drone"] + timeout
  step_counts["agent_7_def_drone"] = step_counts["agent_7_def_drone"] + step_count

  success, timeout, step_count = agent.agent_7_defective_account(env, 150)
  if success:
    wins["agent_7_def_acc_drone"] = wins["agent_7_def_acc_drone"] + 1
  if timeout:
    timeouts["agent_7_def_acc_drone"] = timeouts["agent_7_def_acc_drone"] + timeout
  step_counts["agent_7_def_acc_drone"] = step_counts["agent_7_def_acc_drone"] + step_count

  success, timeout, step_count = agent.agent_8(env, 150)
  if success:
    wins["agent_8"] = wins["agent_8"] + 1
  if timeout:
    timeouts["agent_8"] = timeouts["agent_8"] + timeout
  step_counts["agent_8"] = step_counts["agent_8"] + step_count

  success, timeout, step_count = agent.agent_8_defective_no_account(env, 150)
  if success:
    wins["agent_8_def_drone"] = wins["agent_8_def_drone"] + 1
  if timeout:
    timeouts["agent_8_def_drone"] = timeouts["agent_8_def_drone"] + timeout
  step_counts["agent_8_def_drone"] = step_counts["agent_8_def_drone"] + step_count

  success, timeout, step_count = agent.agent_8_defective_account(env, 150)
  if success:
    wins["agent_8_def_acc_drone"] = wins["agent_8_def_acc_drone"] + 1
  if timeout:
    timeouts["agent_8_def_acc_drone"] = timeouts["agent_8_def_acc_drone"] + timeout
  step_counts["agent_8_def_acc_drone"] = step_counts["agent_8_def_acc_drone"] + step_count

  success, timeout, step_count = agent.bonus_agent(env, 100)
  if success:
    wins["bonus_agent"] = wins["bonus_agent"] + 1
  if timeout:
    timeouts["bonus_agent"] = timeouts["bonus_agent"] + timeout
  step_counts["bonus_agent"] = step_counts["bonus_agent"] + step_count

  success, timeout, step_count = agent.agent_9(env, 100)
  if success:
    wins["agent_9"] = wins["agent_9"] + 1
  if timeout:
    timeouts["agent_9"] = timeouts["agent_9"] + timeout
  step_counts["agent_9"] = step_counts["agent_9"] + step_count

for key in step_counts.keys():
  if wins[key] == 0:
    continue
  step_counts[key] = step_counts[key]/wins[key]

In [ ]:
for k in wins.keys():
  if 'acc' in k:
    worker = k
    worker = worker.replace("_def_acc_drone", " defective drone accounting")
  elif 'def' in k:
    worker = k
    worker = worker.replace("_def_drone", " defective drone scenario")
  else:
    worker = k
  print("Analysis of ", worker)
  print("Success rate: ", (wins[k]/3000)*100)
  print("Timeout rate: ", k, (timeouts[k]/3000)*100)
  print("Average stepcount :", step_counts[k])
  if k in prey_certainity:
    print("Prey ceratinity :", prey_certainity[k]/3000)
  if k in predator_certainity:
    print("Predator certainity :", predator_certainity[k]/3000)
  print("****************************************************************************")

Analysis of  agent_1
Success rate:  92.5
Timeout rate:  agent_1 0.0
Average stepcount : 11.508108108108107
****************************************************************************
Analysis of  agent_2
Success rate:  99.7
Timeout rate:  agent_2 0.06666666666666667
Average stepcount : 11.771648278167836
****************************************************************************
Analysis of  agent_3
Success rate:  85.3
Timeout rate:  agent_3 0.36666666666666664
Average stepcount : 25.819460726846426
Prey ceratinity : 1.242
****************************************************************************
Analysis of  agent_4
Success rate:  96.83333333333334
Timeout rate:  agent_4 2.933333333333333
Average stepcount : 27.950774526678142
Prey ceratinity : 2.0763333333333334
****************************************************************************
Analysis of  agent_5
Success rate:  83.2
Timeout rate:  agent_5 0.03333333333333333
Average stepcount : 18.22676282051282
Predator certainity : 